In [1]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\n\n<style>\n\n@font-face {\n    font-family: \"Computer Modern\";\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\n}\n\n.container { width: 100% }\n\n/* Formatting for header cells */\n.text_cell_render h1 {\n    font-family: 'Philosopher', sans-serif;\n    font-weight: 400;\n    font-size: 2.2em;\n    line-height: 100%;\n    color: rgb(0, 80, 120);\n    margin-bottom: 0.1em;\n    margin-top: 0.1em;\n    display: block;\n}\t\n.text_cell_render h2 {\n    font-family: 'Philosopher', serif;\n    font-weight: 400;\n    font-size: 1.9em;\n    line-height: 100%;\n    color: rgb(200,100,0);\n    margin-bottom: 0.1em;\n    margin-top: 0.1em;\n    display: block;\n}\t\n\n.text_cell_render h3 {\n    font-family: 'Philosopher', serif;\n    margin-top:12px;\n    margin-bottom: 3px;\n    font-style: italic;\n    color: rgb(94,127,192);\n}\n\n.text_cell_render h4 {\n    font-family: 'Philosopher', serif;\n}\n\n.text_cell_render h5 {\n    font-family: 'Alegreya Sans', sans-serif;\n    font-weight: 300;\n    font-size: 16pt;\n    color: grey;\n    font-style: italic;\n    margin-bottom: .1em;\n    margin-top: 0.1em;\n    display: block;\n}\n\n.text_cell_render h6 {\n    font-family: 'PT Mono', sans-serif;\n    font-weight: 300;\n    font-size: 10pt;\n    color: grey;\n    margin-bottom: 1px;\n    margin-top: 1px;\n}\n\n.text_cell_render em {\n    font-family: 'Philosopher', sans-serif;\n    color:        blue;\n    background-color: rgb(255,220,180);\n    font-size:    110%;\n    margin-left:   2px;\n    margin-right:  2px;\n    font-weight:   100;\n}\n\n.text_cell_render b {\n    color:            rgb(255,195,195);\n    background-color: rgb(0,0,0);\n    font-size:    110%;\n    margin-left:   2px;\n    margin-right:  2px;\n    font-weight:   650;\n}\n\n.text_cell_render tt {\n    font-size:    120%;\n    margin-left:   2px;\n    margin-right:  2px;\n    font-weight:   150;\n}\n\n.Codemirror {\n    font-family: \"PT Mono\";\n    font-size: 100%;\n}\n\n</style>")

## Basic Game

Das Modul `Chess.jl` stellt eine Implementation des Schachspiels für die Sprache Julia bereit. Dieses kann gegebenenfalls über den Packagemanager `Pkg` installiert werden.

In [2]:
# using Pkg
# Pkg.add("Chess")

In [3]:
using Chess

Ein Schachspiel besteht aus einem abwechselnden Durchführen jeweils genau eines Zuges. Besondere Konventionen wie zum Beispiel Rochaden zählen hierbei als ein Zug.

Hierbei folgt das Spielen eines Schachspiels stets der folgenden Logik:

ai_turn = false

while !isterminal(game)

    move = next_move(game, ai_turn ? ai : player)
    domove!(game, move)
    ai_turn = !ai_turn
    
end     

Die Funktion `next_move` : (`board`, `entity`) -> `move` bildet auf den nächsten durchzuführenden Zug ab. Hierbei wird die Implementation der Funktion über Multiple-Dispatch mit dem `entity` Attribut bestimmt. Dieses welchselt mit der Partei am Zug.

### Entity Player

Die Spieler-Implementation (Methode) der `next_move` Funktion setzt einer Benutzer-Eingabe voraus, welche in Julia mithilfe der `readline`(`stream`) Funktion erreicht werden kann. 

In [4]:
input = readline(stdin)
print(input)

stdin> exit
exit

Wie bereits zu erkennen ist, kann der Benutzer jeden beliebigen Text als Eingabe vormulieren, auch solche, die nicht einer validen Zugeingabe entsprechen. Um einer solchen fehlerhaften Eingabe entgegen zu wirken, definieren wir die folgende Funktion:

In [5]:
function validate_userinput(input::String)
    if match(r"^[a-hA-H][1-8][a-hA-H][1-8]$", input) === nothing
        return "Invalid input '$(input)'"
    end
    return nothing
end

validate_userinput (generic function with 1 method)

Der reguläre Ausdruck `^[a-h][1-8][a-h][1-8]$` akzeptiert hierbei nur Eingabetexte, welche aus zwei Feldern bestehen: Start & Ende. Da es sich bei der X- und Y-Indizierung auf einem Schachbrett um Koordinaten im Bereich [a-h] und [a-8] handelt, werden diese zusätzlich auf die entsprechenden Bereiche begrenzt. Je nach Status des Spiels ist eine Angabe des Startfelden theoretisch redundant, jedoch entschieden wir uns, die verbose Form als Eingabenorm beizubehalten.

Falls die Eingabe der `validate_userinput` Funktion einen syntaktisch validen Zug aufweist, so gibt diese `nothing` zurück. Der Rückgabewert ist als eine Art Felher zu interpretieren. Ist die Eingabe invalid, so wird ein String mit der Fehlernachricht zurückgegeben.

Des weiteren ist zu prüfen, ob es sich bei dem syntaktisch validen Zug um einen tatsächlich legalen und durchführbaren Zug handelt. Beispielsweise könnte der Benutzer in der Startposition den Zug `"e2f2"` eingeben, welcher dem Bauer instruktieren würde, um ein Feld nach rechts zu gehen. Dieser Zug ist laut Spielregeln nicht möglich und muss daher unterbunden werden. Hierzu definieren wir die folgende Funktion:

In [6]:
function validate_legal_move(board::Board, move::Move)
    all_legal_moves = moves(board)
    if move ∉ all_legal_moves
        return("Illegal move '$(move)'")
    end
    return nothing
end

validate_legal_move (generic function with 1 method)

Mithilfe der `moves(board)` Funktion können wir alle legalen Züge auflisten. Anschließend ist zu prüfen, ob sich der Zugkandidat in der entsprechenden Liste befindet. In Julia kann dies durch die respektiven Operatoren `∈` und `∉` durchgeführt werden. Die `validate_legal_move` Funktion gibt bei einem Fehler ebenfalls einen String zurück, andersweitig `nothing`.

Mit den beiden Prüffunktionen können wir anschließend die `next_move` Funktion für einen Player definieren:

In [7]:
struct Player end

function next_move(board, _::Player)
    move = nothing
    while true
        userinput = readline(stdin)
        if userinput == "exit" || userinput == "reset"
            return userinput
        end
        error = validate_userinput(userinput)
        if error != nothing
            println(error)
            continue
        end
        move = movefromstring(userinput)
        error = validate_legal_move(board, move)
        if error != nothing
            println(error)
            continue
        end
        break
    end
    return move
end

next_move (generic function with 1 method)

Als zusätzlich valide Eingaben definieren wir die Worte `exit` und `reset`, welche gegebenenfalls das Spiel vorzeitig beenden oder zurücksetzten können.

## Game Implementation

Mit allen bereits implementieren Komponenten lässt sich eine erste, lauffähige Durchführung eines Schachspiels implementieren:

In [8]:
function play_game(ai, player = Player(), game = SimpleGame(), ai_turn = false)
    print_game(game)
    
    while !isterminal(game)
        move = next_move(board(game), ai_turn ? ai : player)
        if move == "exit"
            IJulia.clear_output()
            return board(game)
        end
        if move == "reset"
            return play_game(ai)
        end
        domove!(game, move)
        print_game(game)
        ai_turn = !ai_turn
    end
    
    return isdraw(game) ? "Draw!" : ai_turn ? "Checkmate! You win!" : "Checkmate! You lost!"
end

play_game (generic function with 4 methods)

Nachdem ein Spiel beendet wurde, gibt die Funktion das Ergebnis es Spiels zurück.

Um die Ausgabe während des Spiels zu vereinfachen, definieren wir die folgende Funktion, welche die aktuelle Ausgabezelle löscht und das Spielbrett erneut ausgibt. Konvertiert man ein `Chess.jl`- Schachbrett zu HTML, so wird dieses in einer anschaulichen Form ausgerendert.

In [9]:
function print_game(game)
    IJulia.clear_output()
    html = HTML(IJulia.html(board(game)))
    IJulia.display(html)
end

print_game (generic function with 1 method)

Als letzte Komponente für ein vollständiges Schachspiel fehlt ein KI-Gegner. Hierzu importieren wir die `RandomAI`.

In [10]:
# using Pkg
# Pkg.add("NBInclude")

In [11]:
using NBInclude
@nbinclude("RandomAI.ipynb")

next_move (generic function with 2 methods)

In [14]:
play_game(RandomAI())

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R